<a href="https://colab.research.google.com/github/DIFACQUIM/Cursos/blob/main/4_Espacio_Qu%C3%ADmico_t_SNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 36.6 MB/s eta 0:00:00


In [ ]:
!pip install molplotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/39/40/c0d86942ba668d975570a0fbd7fe4224445198a90a64ec6f0c1cd3bf2527/molplotly-1.1.8.tar.gz (from https://pypi.org/simple/molplotly/): Requested molplotly from https://files.pythonhosted.org/packages/39/40/c0d86942ba668d975570a0fbd7fe4224445198a90a64ec6f0c1cd3bf2527/molplotly-1.1.8.tar.gz has inconsistent version: expected '1.1.8', but metadata has '1.1.7'
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00
  Created wheel for molplotly: filename=molplotly-1.1.7-py3-none-any.whl size=14048 sha25

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys, AllChem
from scipy.spatial.distance import pdist

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Pre-simposio QF/4.Espacio_quimico/Dataset.csv")
data.head(2)

,ID,SMILES,Data set,HBA,HBD,RB,LogP,TPSA,MW
0,DB00006,CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=...,FDA,29,27,66,-8.3261,904.07,2180.317
1,DB00007,CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)...,FDA,14,15,32,-1.4381,431.54,1209.421


In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
data_tsne = data.copy()
data_tsne = data_tsne.drop(labels = ["Data set", "ID","SMILES"],axis = 1)
data_tsne = StandardScaler().fit_transform(data_tsne)
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data_tsne)
tsne_results

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 3231 samples in 0.005s...
[t-SNE] Computed neighbors for 3231 samples in 0.196s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3231
[t-SNE] Computed conditional probabilities for sample 2000 / 3231
[t-SNE] Computed conditional probabilities for sample 3000 / 3231
[t-SNE] Computed conditional probabilities for sample 3231 / 3231
[t-SNE] Mean sigma: 0.199656
[t-SNE] KL divergence after 250 iterations with early exaggeration: 71.133118
[t-SNE] KL divergence after 300 iterations: 1.785134


array([[11.040099  , -0.95313317],
       [10.156716  , -0.65292484],
       [10.318821  , -0.61155236],
       ...,
       [ 4.6015334 , -7.3071    ],
       [ 4.655387  , -7.000716  ],
       [ 1.7202786 , -5.6627812 ]], dtype=float32)

In [ ]:
label = data[["Data set", "ID", "SMILES"]]
label = label.to_numpy()
label

array([['FDA', 'DB00006',
        'CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(=O)C(CC(=O)O)NC(=O)CNC(=O)C(CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C1CCCN1C(=O)C(N)Cc1ccccc1)C(=O)N1CCCC1C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CC(C)C)C(=O)O'],
       ['FDA', 'DB00007',
        'CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CO)NC(=O)C(Cc1c[nH]c2ccccc12)NC(=O)C(Cc1c[nH]cn1)NC(=O)C1CCC(=O)N1'],
       ['FDA', 'DB00014',
        'CC(C)CC(NC(=O)C(COC(C)(C)C)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CO)NC(=O)C(Cc1c[nH]c2ccccc12)NC(=O)C(Cc1c[nH]cn1)NC(=O)C1CCC(=O)N1)C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NNC(N)=O'],
       ...,
       ['DNMT1', '"CHEMBL4866673',
        'COc1cc2c(=NCc3ccc(-c4ccc(C(O)=NO)cc4)cc3)cc(-c3ccc(C)o3)[nH]c2cc1OCCCN1CCCC1'],
       ['DNMT1', '"CHEMBL4878570',
        'ON=C(O)C1CCN(CC(O)COC(Cn2c3ccccc3c3ccccc32)Cn2c3ccccc3c3ccccc32)CC1'],
       ['DNMT1', '"CHEMBL4454

In [ ]:
tsne_results

array([[11.040099  , -0.95313317],
       [10.156716  , -0.65292484],
       [10.318821  , -0.61155236],
       ...,
       [ 4.6015334 , -7.3071    ],
       [ 4.655387  , -7.000716  ],
       [ 1.7202786 , -5.6627812 ]], dtype=float32)

In [ ]:
# Concatenar arrays de numpy
arr = np.concatenate((label, tsne_results), axis = 1)
arr.shape

(3231, 5)

In [ ]:
# Crear un nuevo dataframe
tsne_dataset = pd.DataFrame(data=arr, columns = ['Data set',"ID", "SMILES",'axis 1', 'axis 2'] )
tsne_dataset.head(2)

,Data set,ID,SMILES,axis 1,axis 2
0,FDA,DB00006,CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=...,11.040099,-0.953133
1,FDA,DB00007,CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)...,10.156716,-0.652925


In [ ]:
def color(label):
  if label == str("BIOFACQUIM"):
    return int(0)
  if label == str("FDA"):
    return int(1)
  if label == str("DNMT1"):
    return int(2)

In [ ]:
tsne_dataset["color"] = [color(x) for x in tsne_dataset["Data set"]]
tsne_dataset.head(2)

,Data set,ID,SMILES,axis 1,axis 2,color
0,FDA,DB00006,CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=...,11.040099,-0.953133,1
1,FDA,DB00007,CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)...,10.156716,-0.652925,1


In [ ]:
import plotly.express as px

fig = px.scatter(tsne_dataset, x="axis 1", y="axis 2", color="color",
                 color_continuous_scale=['orange', "indigo", "green"]
                 )
import molplotly
# add molecules to the plotly graph - returns a Dash app
app = molplotly.add_molecules(fig=fig,
                            df=tsne_dataset,
                            smiles_col='SMILES',
                            title_col='Data set',
                            color_col="ID"
                            )

# run Dash app inline in notebook (or in an external server)
app.run_server(mode='inline', port=8700, height=1000)

Dash is running on http://127.0.0.1:8700/



INFO:dash.dash:Dash is running on http://127.0.0.1:8700/



<IPython.core.display.Javascript object>

In [ ]:
#===============================================================================